# Examining problematic sulfonamide geometries

In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies
import multiprocessing
import tqdm
import numpy as np
import sys
import openmm
from openff.units.openmm import from_openmm
import os
# suppress stereochemistry warnings
import logging
from rdkit import Chem
import itertools

from openff.qcsubmit.results import TorsionDriveResultCollection, OptimizationResultCollection
import logging
logging.getLogger("openff").setLevel(logging.ERROR)

from openff.qcsubmit.results.filters import SMARTSFilter

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms

In [2]:
def minimize_energy(mol,ff):
    topo = mol.to_topology()
    interchange = Interchange.from_smirnoff(force_field=ff, topology=topo,allow_nonintegral_charges=True)#,charge_from_molecules=[mol]) #,charge_from_molecules=[mol])
    integrator = openmm.VerletIntegrator(1 * openmm.unit.femtoseconds)
    simulation = interchange.to_openmm_simulation(integrator)

    # We'll store energies in two lists
    initial_energies = []
    minimized_energies = []

    # And minimized conformers in a second molecule
    minimized_molecule = Molecule.from_topology(topo)
    minimized_molecule.conformers.clear()

    conformer = mol.conformers[0]
    # Tell the OpenMM Simulation the positions of this conformer
    simulation.context.setPositions(conformer.to_openmm())

    # Keep a record of the initial energy
    initial_energies.append(
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
    )

    # Perform the minimization
    simulation.minimizeEnergy()#tolerance=Quantity(value=0.0026255, unit=kilojoule/mole))#tolerance=5e-9)

    # Record minimized energy and positions
    min_state = simulation.context.getState(getEnergy=True, getPositions=True)

    minimized_energies.append(min_state.getPotentialEnergy())
    minimized_molecule.add_conformer(from_openmm(min_state.getPositions()))
    return initial_energies,minimized_energies,minimized_molecule


# Finding test molecules from benchmarking dataset
Filter for N-S(=O)(=O)-R (where R != N)

In [8]:
industry_benchmark = OptimizationResultCollection.parse_file('../../datasets/filtered-industry.json')

In [51]:
industry_benchmark_filter = industry_benchmark.filter(SMARTSFilter(smarts_to_include=['[#7]~[#16X4:1](=[#8])(-[#8])=[#8]']))

In [52]:
industry_benchmark_filter.n_results

88

In [53]:
industry_benchmark_rm = industry_benchmark_filter.to_records()

Choosing a few problem cases:

# Molecule 38, QCArchive ID 36973709
Bad with Sage 2.0.0 and 2.1, fixed in 2.2

In [72]:
# test_mol1 = industry_benchmark_rm[28][1]
# test_mol1

NGLWidget()

In [73]:
# test_mol1.to_file('qcaid_36973709.sdf',file_format='sdf')

In [3]:
test_mol1 = Molecule.from_file('qcaid_36973709.sdf')
test_mol1

NGLWidget()

In [4]:
ie_sage200_qm1,me_sage200_qm1,min_mol_sage200_qm1 = minimize_energy(test_mol1,ForceField('openff_unconstrained-2.0.0.offxml'))

In [5]:
min_mol_sage200_qm1

NGLWidget()

In [6]:
ie_sage210_qm1,me_sage210_qm1,min_mol_sage210_qm1 = minimize_energy(test_mol1,ForceField('openff_unconstrained-2.1.0.offxml'))

In [7]:
min_mol_sage210_qm1

NGLWidget()

In [8]:
ie_sage220_qm1,me_sage220_qm1,min_mol_sage220_qm1 = minimize_energy(test_mol1,ForceField('../../../openff_unconstrained-2.2.0-rc1.offxml',allow_cosmetic_attributes=True))

In [9]:
min_mol_sage220_qm1

NGLWidget()

In [10]:
min_mol_sage220_qm1.to_file('qcaid_36973709_optsage220.sdf',file_format='sdf')

In [11]:
opt1_sage200 = min_mol_sage200_qm1.to_rdkit()
opt1_sage200_conf = opt1_sage200.GetConformer()

In [12]:
opt1_sage210 = min_mol_sage210_qm1.to_rdkit()
opt1_sage210_conf = opt1_sage210.GetConformer()

In [30]:
opt1_sage220 = min_mol_sage220_qm1.to_rdkit()
opt1_sage220_conf = opt1_sage220.GetConformer()

In [14]:
qm_rdkit1 = test_mol1.to_rdkit()
qm_conf1 = qm_rdkit1.GetConformer()

### Geometries

In [15]:
atoms1=[a for a in opt1_sage200.GetAtoms()]

In [16]:
atom_indices_s1 = [a.GetIdx() for a in atoms1 if a.GetSymbol() == 'S']

In [17]:
atom_indices_n1 = [a.GetIdx() for a in atoms1 if a.GetSymbol()=='N']

In [18]:
atom_indices_o1 = [a.GetIdx() for a in atoms1 if a.GetSymbol()=='O']

In [19]:
print(atom_indices_s1)
print(atom_indices_n1)
print(atom_indices_o1)

[18]
[21]
[3, 4, 7, 11, 14, 17, 19, 20]


In [20]:
s = atom_indices_s1[0]
n1 = atom_indices_n1[0]
o1,o2,o3 = atom_indices_o1[5:]

In [21]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,n1) # N

1.6251460038597085

In [22]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,o3) # N

1.461543281478046

In [23]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,o1) # O

1.6600324911582192

In [24]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s ,o2) # O

1.4651177968606206

In [25]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o3)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 98.72643446313526
N1 - S - O1: 98.61532656209359
N1 - S - O2: 99.11770120565771
O1 - S - O2: 117.29281127704672


In [26]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o3)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 87.66103608848968
N1 - S - O1: 100.1410615790904
N1 - S - O2: 89.15854398351553
O1 - S - O2: 117.02802853954255


In [31]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o3)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 99.28537645791587
N1 - S - O1: 105.15074149609262
N1 - S - O2: 101.6658123656726
O1 - S - O2: 111.30402335350927


In [95]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o3)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 106.32447550720543
N1 - S - O1: 96.99188306450321
N1 - S - O2: 111.58892836214494
O1 - S - O2: 107.38049316326159
